<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-12.-Inheritance:-For-Good-or-For-Worse" data-toc-modified-id="Chapter-12.-Inheritance:-For-Good-or-For-Worse-1">Chapter 12. Inheritance: For Good or For Worse</a></span><ul class="toc-item"><li><span><a href="#Subclassing-Built-In-Types-Is-Broken" data-toc-modified-id="Subclassing-Built-In-Types-Is-Broken-1.1">Subclassing Built-In Types Is Broken</a></span></li><li><span><a href="#Method-Resolution-Order" data-toc-modified-id="Method-Resolution-Order-1.2">Method Resolution Order</a></span><ul class="toc-item"><li><span><a href="#The-diamond-problem-with-multiple-inheritence" data-toc-modified-id="The-diamond-problem-with-multiple-inheritence-1.2.1">The diamond problem with multiple inheritence</a></span></li><li><span><a href="#Inheritance-of-interface" data-toc-modified-id="Inheritance-of-interface-1.2.2">Inheritance of interface</a></span></li><li><span><a href="#Inheritance-of-implementation" data-toc-modified-id="Inheritance-of-implementation-1.2.3">Inheritance of implementation</a></span></li></ul></li></ul></li></ul></div>

# Chapter 12. Inheritance: For Good or For Worse

## Subclassing Built-In Types Is Broken
special methods are never called by other built-in methods of the same object.

Subclassing built-in types like dict or list or str directly is error-prone because the built-in methods mostly ignore user-defined overrides. 

In [440]:
class DoppelDict(dict):
     def __setitem__(self, key, value):
        print('__setitem__ called') 
        super().__setitem__(key, [value] *2)

In [441]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [442]:
dd['two'] = 2
dd

__setitem__ called


{'one': 1, 'two': [2, 2]}

In [443]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

Instead of subclassing the built-ins, derive your classes from the collections module using UserDict, UserList, and UserString, which are designed to be easily extended

In [444]:
import collections

class DoppelDict(collections.UserDict):
     def __setitem__(self, key, value):
        print('__setitem__ called') 
        super().__setitem__(key, [value] *2)

In [445]:
dd = DoppelDict(one=1)
dd

__setitem__ called


{'one': [1, 1]}

In [446]:
dd['two'] = 2
dd

__setitem__ called


{'one': [1, 1], 'two': [2, 2]}

In [447]:
dd.update(three=3)
dd

__setitem__ called


{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

These problems apply only to method delegation within the C language implementation of the built-in types, and only affects user-defined classes derived directly from those types. 

If you subclass from a class coded in Python, such as UserDict or MutableMapping, you will not be troubled by this

## Method Resolution Order

### The diamond problem with multiple inheritence

In [448]:
class A:
    def ping(self):
        print('calling A:', self)


# t both classes B and C implement a pong method. 
# The only difference is that C.pong outputs the word 
# PONG in uppercase.
class B(A):
    def pong(self):
        print('calling B :', self)

class C(A):
    def pong(self):
        print('calling C:', self)


class D(B, C):

    def ping(self):
        super().ping()
        print('calling D:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


In [450]:
#  B.pong is always called because of the Method Resolution Order
d = D()
d.pong()

calling B : <__main__.D object at 0x7f8a03a7bf10>


the Method Resolution Order `__mro__` attribute gives the calling order up the inheritence hierarchy

In [451]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [452]:
def print_mro(cls):  
     print(', '.join(c.__name__ for c in cls.__mro__))

In [453]:
print_mro(D)

D, B, C, A, object


You can always call a method on a superclass directly, passing the instance as an explicit argument.

In [454]:
C.pong(d)

calling C: <__main__.D object at 0x7f8a03a7bf10>


### Inheritance of interface
creates a subtype, implying an “is-a” relationship.

In [456]:
d.pingpong()

calling A: <__main__.D object at 0x7f8a03a7bf10>
calling D: <__main__.D object at 0x7f8a03a7bf10>
calling A: <__main__.D object at 0x7f8a03a7bf10>
calling B : <__main__.D object at 0x7f8a03a7bf10>
calling B : <__main__.D object at 0x7f8a03a7bf10>
calling C: <__main__.D object at 0x7f8a03a7bf10>


### Inheritance of implementation 
avoids code duplication by reuse.